In [75]:
import pandas as pd
import numpy as np

In [76]:
r_e = 2.817e-15
c = 3e8
n = 6.9e10
e = 1.6e-19
q=n*e
E_e = 150
mc2 = 0.511
gamma = E_e/mc2
pi = 3.1416
C = 171.7
sigma_s = 0.025
gamma

293.5420743639922

In [77]:
A = r_e*r_e*c*q/(8*pi*e*gamma*gamma*gamma*sigma_s*C)
A

6.019796132036648e-20

In [78]:
df_ = pd.read_csv("tp0.txt",delim_whitespace=True)
s = df_['s']
df_ = (s - np.roll(s, 1))[1:]# second row - first row, 1: removes the first row at the end
df_.reset_index(drop = True, inplace=True)
df_
df0 = pd.DataFrame(df_) #Converting into dataframe
df0

,s
0,1.40
1,0.25
2,2.90
3,0.25
4,2.90
...,...
158,2.90
159,0.25
160,2.90
161,0.25


In [79]:
# Load all twiss parameters and momentum aperture
df1=pd.read_csv("tp1.txt",sep="\s+")
df2=pd.read_csv("tp2.txt",sep="\s+")
df3=pd.read_csv("ma2.txt",sep="\s+")

In [80]:
# Merge df1 df2 and df3
df4 = df0.reset_index().merge(df1.reset_index(), left_index=True, right_index=True, how='left')
df5 = df2.reset_index().merge(df3.reset_index(), left_index=True, right_index=True, how='left')
df = df4.reset_index().merge(df5.reset_index(), left_index=True, right_index=True, how='left')

In [81]:
df

,index_x,index_x_x,s,index_y_x,bx,ax,by,ay,sigx1,sigy,gx,index_y,index_x_y,etx,etxp,index_y_y,dPos,dNeg
0,0,0,1.40,0,3.284000,-5.600000e-01,3.284000,-5.600000e-01,0.000057,0.001690,0.400000,0,0,-2.134721e-10,-6.527099e-11,0,0.009,-0.007300
1,1,1,0.25,1,3.842075,-1.724942e+00,3.347376,3.125298e-01,0.000066,0.001707,1.034708,1,1,-2.376159e-10,-1.290226e-10,1,0.009,-0.007211
2,2,2,2.90,2,22.548640,-4.725596e+00,4.292519,-6.384410e-01,0.000390,0.001932,1.034708,2,2,-6.117814e-10,-1.290226e-10,2,0.009,-0.007211
3,3,3,0.25,3,23.187460,2.235301e+00,4.983429,-2.194795e+00,0.000401,0.002082,0.258613,3,3,-6.205507e-10,5.931257e-11,3,0.009,-0.007300
4,4,4,2.90,4,12.397650,1.485324e+00,27.530180,-5.579946e+00,0.000214,0.004894,0.258613,4,4,-4.485443e-10,5.931257e-11,4,0.009,-0.007300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,158,158,2.90,158,23.187460,-2.235301e+00,4.983429,2.194795e+00,0.000401,0.002082,0.258613,158,158,-6.205502e-10,-5.931260e-11,158,0.009,-0.006298
159,159,159,0.25,159,22.548640,4.725596e+00,4.292519,6.384410e-01,0.000390,0.001932,1.034708,159,159,-6.117810e-10,1.290224e-10,159,0.009,-0.006408
160,160,160,2.90,160,3.842075,1.724942e+00,3.347376,-3.125298e-01,0.000066,0.001707,1.034708,160,160,-2.376160e-10,1.290224e-10,160,0.009,-0.006408
161,161,161,0.25,161,3.284000,5.600000e-01,3.284000,5.600000e-01,0.000057,0.001690,0.400000,161,161,-2.134722e-10,6.527079e-11,161,0.009,-0.006300


In [82]:
sig_d = 3.03E-03
sig_d_sq = sig_d*sig_d

In [83]:
for ind, row in df.iterrows():
    df.loc[ind,"etx_del_sq"] = sig_d_sq * (row["etx"]*row["etx"])
for ind, row in df.iterrows():
    df.loc[ind,"sigx"] = np.sqrt(row["sigx1"] + row["etx_del_sq"])


In [84]:
for ind, row in df.iterrows():
    df.loc[ind,"H(s)"] = (row["gx"]*row["etx"]*row["etx"]) + (2* row["ax"]*row["etx"]*row["etxp"]) + (row["bx"]*row["etxp"]*row["etxp"])

In [85]:
#ex = 18.20e-06
ex = 17.30E-06
sig_d = 3.03E-03
sig_d_sq = sig_d*sig_d
ratio = sig_d_sq / ex
for ind, row in df.iterrows():
    df.loc[ind,"sig_div"] = (ex /row["sigx"])*np.sqrt(1 + (row["H(s)"]*ratio))

In [86]:
gamma_L = 293.54
for ind, row in df.iterrows():
    df.loc[ind,"denom"] = row["sig_div"]*gamma_L
    
for ind, row in df.iterrows():
    df.loc[ind,"x"] = (row["dPos"]/row["denom"])*(row["dPos"]/row["denom"])

In [87]:
# Integrating F(x) and getting array values of integration for all values of x
from scipy.integrate import quad
import math
from math import log
x_values = df["x"].to_numpy()
I1 = np.empty(len(x_values))
def f(u,x):
    return (1/u - 0.5*np.log(1/u) - 1)*np.exp(-x/u)
for i, x in enumerate(x_values):
    I1[i] = quad(f, 0, 1, args=(x))[0]
I1

array([6.55957645, 6.40339276, 4.65372381, 4.62637219, 5.24193814,
       5.22653563, 4.37481177, 4.37878204, 5.37978786, 5.4451662 ,
       5.59487708, 5.62368709, 6.14720499, 6.14054837, 5.89462177,
       5.83629649, 5.61504948, 5.63314361, 6.06660241, 6.92881466,
       7.03204281, 6.98025312, 6.46929912, 6.01193221, 5.51474236,
       5.50688821, 5.90748635, 6.49524792, 6.9809675 , 6.98755841,
       6.89452785, 6.79790041, 6.5018005 , 6.33160224, 6.0794898 ,
       6.07948973, 6.13008311, 6.18073708, 6.33160218, 6.84266159,
       7.32885553, 7.32561969, 7.23251378, 7.13582549, 6.83942466,
       5.9908377 , 5.73920988, 5.73218332, 5.78545539, 5.83319883,
       5.98658962, 6.29543462, 6.78081264, 6.78081264, 6.29543462,
       5.95133735, 5.41696618, 5.55671798, 6.05681571, 6.5715331 ,
       7.27509831, 7.29355235, 6.94740072, 6.06660241, 5.63314361,
       5.61504948, 5.83629648, 5.89462177, 6.14054837, 6.21420534,
       6.09157718, 4.71175101, 4.76866858, 9.21659723, 8.98360

In [88]:
for ind, row in df.iterrows():
    df.loc[ind,"dPos_sq"] = row["dPos"]*row["dPos"]
for ind, row in df.iterrows():
    df.loc[ind,"I2"] = row["sigx"]*row["sigy"]*row["sig_div"]*row["dPos_sq"]

In [91]:
df["I1"] = pd.Series(I1) # Convert numpy array into dataframe column


In [92]:
df["I1"].mean()

6.029794589910142

In [94]:
for ind, row in df.iterrows():
    df.loc[ind,"I"] = (row["I1"] / row["I2"])*row["s"]
df["I"].sum()

273992862908617.53

In [96]:
I_sum = df["I"].sum()
print(I_sum)

273992862908617.53


In [97]:
pos_trate = A*I_sum
pos_tlifetime = 1/(pos_trate)
pos_lifetime = pos_tlifetime/3600
pos_lifetime

16.841333086732252

### Touscheck Rate for delta_negative

In [98]:
gamma_L = 293.54
for ind, row in df.iterrows():
    df.loc[ind,"denom"] = row["sig_div"]*gamma_L
    
for ind, row in df.iterrows():
    df.loc[ind,"y"] = (row["dNeg"]/row["denom"])*(row["dNeg"]/row["denom"])

In [99]:
from scipy.integrate import quad
import math
from math import log
y_values = df["y"].to_numpy()
I1y = np.empty(len(y_values))
def f(u,y):
    return (1/u - 0.5*np.log(1/u) - 1)*np.exp(-y/u)
for i, y in enumerate(y_values):
    I1y[i] = quad(f, 0, 1, args=(y))[0]
I1y

array([6.97664855, 6.84472186, 5.08918235, 5.03748068, 5.65595842,
       5.63356969, 4.77741704, 4.78827495, 5.79429211, 5.85988258,
       6.01004061, 6.03893088, 6.5636328 , 6.55696397, 6.31053842,
       6.19055064, 5.96892444, 5.99209212, 6.40718184, 7.05699044,
       7.40322815, 7.31744275, 6.66984272, 6.41880478, 5.90215921,
       5.91079394, 6.31262233, 6.91202324, 7.39830327, 7.39623438,
       7.30311468, 7.20641293, 6.9099567 , 6.25312746, 6.01749191,
       5.97276605, 6.02356742, 6.0741926 , 6.26411149, 6.64919656,
       7.13407429, 7.13699493, 7.04389297, 6.94727978, 6.65103735,
       6.30583549, 6.05375992, 6.06324153, 6.11136356, 6.16449489,
       6.31505373, 6.71211847, 7.19815288, 7.28938456, 6.75616899,
       6.27464332, 5.87354979, 5.87871347, 6.37978214, 6.63049103,
       7.32472753, 7.37898438, 6.99699928, 6.48684389, 6.06759274,
       6.03026939, 6.22503023, 6.41541572, 6.66154083, 6.73627189,
       6.56039839, 5.17535083, 5.29503159, 9.75109564, 9.56337

In [100]:
for ind, row in df.iterrows():
    df.loc[ind,"dNeg_sq"] = row["dNeg"]*row["dNeg"]
for ind, row in df.iterrows():
    df.loc[ind,"I2y"] = row["sigx"]*row["sigy"]*row["sig_div"]*row["dNeg_sq"]

In [101]:
df["I1y"] = pd.Series(I1y) # Convert numpy array into dataframe column

In [105]:
for ind, row in df.iterrows():
    df.loc[ind,"Iy"] = (row["I1y"]/ row["I2y"])*row["s"] # total integration = numerator divide by denominator

In [106]:
Iy_sum = df["Iy"].sum()
print(Iy_sum)

421692337440306.8


In [107]:
neg_trate = A*Iy_sum
neg_tlifetime = 1/(neg_trate)
neg_lifetime = neg_tlifetime/3600
neg_lifetime

10.942586947728433

In [108]:
t_rate = (1/(2*pos_lifetime))+(1/(2*neg_lifetime))
t_rate

0.07538189540716071

In [109]:
tlifetime = 1/t_rate
tlifetime

13.265784769654486